In [1]:
import json

# Let's load Wilhelm's Translation of the I-Ching, and let's poor it into a JSON & CSV file

In `./iching_fulltext.md` we have a full english translation of the Wilhelm I-Ching translation. We want to write a simple parser which will take out all required pieces from the text and store it inside ann appropriately constructed JSON/CSV.

In [4]:
count = 0
with open('./iching_fulltext.md') as f:
    lines = f.readlines()

with open('./i-ching-basic.js') as json_file:
    iching_basic = json.load(json_file)

iching_full_text = ''.join(lines)

def iching_raw_text_section_split():
    start_ii = 10 # hex 1
    end_ii = 74 # hex 64

    iching_raw = {}
    iching_raw['hexagrams'] = {}
    split = iching_full_text.split("## ")

    for i in range(start_ii, end_ii):
        iching_raw['hexagrams'][i-start_ii] = [p.split("\n") for p in split][i]

    return iching_raw

iching_raw = iching_raw_text_section_split()

keywords = [ 'above', 'below', 'THE JUDGMENT', 'THE IMAGE', 'THE LINES' ]

In [12]:
iching_raw['hexagrams'][2]

['3. Chun / Difficulty at the Beginning',
 '[Back to Index](#index)',
 '',
 "above K'AN THE ABYSMAL, WATER",
 '',
 'below CHêN THE AROUSING, THUNDER',
 '',
 'The name of the hexagram, Chun, really connotes a blade of grass pushing against an obstacle as it sprouts out of the earth--hence the meaning, "difficulty at the beginning." The hexagram indicates the way in which heaven and earth bring forth individual beings. It is their first meeting, which is beset with difficulties. The lower trigram Chên is the Arousing; its motion is upward and its image is thunder. The upper trigram K\'an stands for the Abysmal, the dangerous. Its motion is downward and its image is rain. The situation points to teeming, chaotic profusion; thunder and rain fill the air. But the chaos clears up. While the Abysmal sinks, the upward movement eventually passes beyond the danger. A thunderstorm brings release from tension, and all things breathe freely again.',
 '',
 'THE JUDGMENT',
 '',
 'DIFFICULTY AT THE BE

We want to do the following:
- identify lines with above keywords.
- If they are being identified, if all are identified, it's good news
- Then be sure to add binary notation, and PUIR-notation.

In [54]:
def wilhelm_hexagram(iching_hexagrams, idx):
    # get a single iching hexagram
    hexg = iching_hexagrams[idx]

    # create indice arrray where keywords occur
    idx_array = {}

    # iteratie through lines in hexagram
    for i, l in enumerate(hexg):

        # now we're gonna check if the line contains a special keyword
        for k in keywords:
            
            # if keyword in line
            if k in l[:len(k)]:
                idx_array[k] = i


    return hexg, idx_array

Todo:
- Get started by addinng to a dictionary the symbolic description of the hexagram.
- Notice that there is the LAW, and then there's the extra intepretation and explanation of that order.

In [55]:
# print (get_basic_info(a, idx_array)+"\n")
# print (retrieve_above(a, idx_array)+"\n")
# print (retrieve_below(a, idx_array)+"\n")
# print (retrieve_symbolic(a, idx_array)+"\n")
# print (retrieve_judgment(a, idx_array)+"\n")
# print (retrieve_image(a, idx_array)+"\n")
# print (retrieve_lines(a, idx_array)+"\n")

In [56]:
iching_extented_start = {ii:item for ii, item in enumerate(iching_basic) }

# iching_extented_start = 
# {0: {'hex': 1,
#   'hex_font': '䷀',
#   'trad_chinese': '乾',
#   'pinyin': 'qián',
#   'english': 'Initiating',
#   'binary': 111111,
#   'od': '02'},
#  1: {'hex': 2,
#   'hex_font': '䷁',
#   ...
#   }
# }

In [73]:
for kk, hexagram in iching_extented_start.items():
    # retrieve hexagram text on key kk, then  
    print(kk, "series")
    hex_wilhelm, markers = wilhelm_hexagram(iching_raw['hexagrams'], kk)
    print (retrieve_above(hex_wilhelm, markers))
    print (retrieve_below(hex_wilhelm, markers))

    hexagram['wilhelm_above'] = retrieve_above(hex_wilhelm, markers)
    hexagram['wilhelm_below'] = retrieve_below(hex_wilhelm, markers)
    hexagram['wilhelm_symbolic'] = retrieve_symbolic(hex_wilhelm, markers)
    hexagram['wilhelm_judgment'] = retrieve_judgment(hex_wilhelm, markers)
    hexagram['wilhelm_image'] = retrieve_image(hex_wilhelm, markers)
    hexagram['wilhelm_lines'] = retrieve_lines(hex_wilhelm, markers)

    print("")
    if kk == 3:
        break


0 series
above CH'IEN THE CREATIVE, HEAVEN
below CH'IEN THE CREATIVE, HEAVEN

1 series
above K'UN THE RECEPTIVE, EARTH
below K'UN THE RECEPTIVE, EARTH

2 series
above K'AN THE ABYSMAL, WATER
below CHêN THE AROUSING, THUNDER

3 series
above KêN KEEPING STILL, MOUNTAIN
below K'AN THE ABYSMAL, WATER



In [74]:
iching_extented_start[2]

{'hex': 3,
 'hex_font': '䷂',
 'trad_chinese': '屯',
 'pinyin': 'zhūn',
 'english': 'Beginning',
 'binary': '010001',
 'od': 50,
 'above': "above K'AN THE ABYSMAL, WATER",
 'below': 'below CHêN THE AROUSING, THUNDER',
 'wilhelm_above': "above K'AN THE ABYSMAL, WATER",
 'wilhelm_below': 'below CHêN THE AROUSING, THUNDER',
 'wilhelm_symbolic': '\nThe name of the hexagram, Chun, really connotes a blade of grass pushing against an obstacle as it sprouts out of the earth--hence the meaning, "difficulty at the beginning." The hexagram indicates the way in which heaven and earth bring forth individual beings. It is their first meeting, which is beset with difficulties. The lower trigram Chên is the Arousing; its motion is upward and its image is thunder. The upper trigram K\'an stands for the Abysmal, the dangerous. Its motion is downward and its image is rain. The situation points to teeming, chaotic profusion; thunder and rain fill the air. But the chaos clears up. While the Abysmal sinks, th

In [77]:
with open('data.txt', 'w') as outfile:
    json.dump(iching_extented_start, outfile)

In [76]:
with open('iching_extented_start.json', 'w') as outfile:
    json.dump(data, outfile)

NameError: name 'data' is not defined

In [20]:
retrieve_above(a, idx_array)

"above K'UN THE RECEPTIVE, EARTH"

In [60]:
def get_basic_info(hex_text, indices):
    """  """
    return hex_text[0]

def retrieve_above(hex_text, indices):
    """  """
    above_text = hex_text[indices['above']]
    return above_text

def retrieve_below(hex_text, indices):
    """  """
    below_text = hex_text[indices['below']]
    return below_text

def retrieve_symbolic(hex_text, indices):
    """  """
    symbolic_text = '\n'.join( hex_text[indices['below']+1: indices['THE JUDGMENT']])
    return symbolic_text

def retrieve_judgment(hex_text, indices):
    """  """
    judgment_text = '\n'.join( hex_text[indices['THE JUDGMENT']+1: indices['THE IMAGE']])
    return judgment_text

def retrieve_image(hex_text, indices):
    """  """
    image_text = '\n'.join( hex_text[indices['THE IMAGE']+1: indices['THE LINES']])
    return image_text

def retrieve_lines(hex_text, indices):
    """ Retrieve symbolic language, and interpretation"""
    lines_text = '\n'.join( hex_text[indices['THE LINES']+1:])
    return lines_text

In [61]:
iching_raw['hexagrams'][1]

["2. K'un / The Receptive",
 '[Back to Index](#index)',
 '',
 "above K'UN THE RECEPTIVE, EARTH",
 '',
 "below K'UN THE RECEPTIVE, EARTH",
 '',
 'This hexagram is made up of broken lines only. The broken lines represents the dark, yielding, receptive primal power of yin. The attribute of the hexagram is devotion; its image is the earth. It is the perfect complement of THE CREATIVE--the complement, not the opposite, for the Receptive does not combat the Creative but completes it . It represents nature in contrast to spirit, earth in contrast to heaven, space as against time, the female-maternal as against the male-paternal. However, as applied to human affairs, the principle of this complementary relationship is found not only in the relation between man and woman, but also in that between prince and minister and between father and son. Indeed, even in the individual this duality appears in the coexistence of the spiritual world and the world of the senses. But strictly speaking there is